In [1]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))



Using TensorFlow backend.


In [2]:
conv_base.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [3]:
import os, shutil

#Original Dataset
dir_original_dataset = 'C:/Users/RitZ/Desktop/original'

# Create Subet of the dataset used in the activity
dir_main = 'C:/Users/RitZ/Desktop/my_data_subset'
os.mkdir(dir_main)

# directories for test,train and validation

dir_train = os.path.join(dir_main, 'train')
os.mkdir(dir_train)
dir_validation = os.path.join(dir_main, 'validation')
os.mkdir(dir_validation)
dir_test = os.path.join(dir_main, 'test')
os.mkdir(dir_test)

# Directory with  training cat pictures
dir_train_cats = os.path.join(dir_train, 'cats')
os.mkdir(dir_train_cats)

# Directory with  training dog pictures
dir_train_dogs = os.path.join(dir_train, 'dogs')
os.mkdir(dir_train_dogs)

# Directory with  validation cat pictures
dir_validation_cats = os.path.join(dir_validation, 'cats')
os.mkdir(dir_validation_cats)

# Directory with  validation dog pictures
dir_validation_dogs = os.path.join(dir_validation, 'dogs')
os.mkdir(dir_validation_dogs)

# Directory with  validation cat pictures
dir_test_cats = os.path.join(dir_test, 'cats')
os.mkdir(dir_test_cats)

# Directory with  validation dog pictures
dir_test_dogs = os.path.join(dir_test, 'dogs')
os.mkdir(dir_test_dogs)

# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(dir_original_dataset, fname)
    dst = os.path.join(dir_train_cats, fname)
    shutil.copyfile(src, dst)

# Copy  500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(dir_original_dataset, fname)
    dst = os.path.join(dir_validation_cats, fname)
    shutil.copyfile(src, dst)
    
# Copy  500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(dir_original_dataset, fname)
    dst = os.path.join(dir_test_cats, fname)
    shutil.copyfile(src, dst)
    
# Copy  1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(dir_original_dataset, fname)
    dst = os.path.join(dir_train_dogs, fname)
    shutil.copyfile(src, dst)
    
# Copy  500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(dir_original_dataset, fname)
    dst = os.path.join(dir_validation_dogs, fname)
    shutil.copyfile(src, dst)
    
# Copy 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(dir_original_dataset, fname)
    dst = os.path.join(dir_test_dogs, fname)
    shutil.copyfile(src, dst)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:/Users/RitZ/Desktop/my_data_subset'

In [ ]:
import os 
import numpy as np 
from keras.preprocessing.image import ImageDataGenerator

dir_main = 'C:/Users/RitZ/Desktop/my_data_subset'
dir_train = os.path.join(dir_main, 'train')
dir_validation = os.path.join(dir_main, 'validation')
dir_test = os.path.join(dir_main, 'test')


In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20



In [ ]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels


In [ ]:
train_features, train_labels = extract_features(dir_train, 2000) 
validation_features, validation_labels = extract_features(dir_validation, 1000)
test_features, test_labels = extract_features(dir_test, 1000)

In [ ]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
conv_base.trainable = False


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
conv_base.summary()

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)